## 데이터 로더(Data Loaders)

이 노트북은 `pyTigerGraph` 에서 **edge neighbor loader** 의 사용을 보여줍니다. 데이터 로더의 역할은 `TigerGraph` 데이터베이스에서 데이터를 가져오는 것입니다. 현재 다음 데이터 로더가 제공됩니다.

* EdgeLoader, 간선의 배치(batch)를 반환합니다.
* VertexLoader, 정점의 배치(batch)를 반환합니다.
* GraphLoader, 무작위로 샘플링된(연결이 끊긴) 하위 그래프(subgraphs)를 pandas의 `dataframe`, `PyG` 또는 `DGL` 형식으로 반환 합니다.
* NeighborLoader, 이웃 샘플링을 사용하여 하위 그래프(subgraphs)를 `dataframe`, `PyG` 또는 `DGL`  형식으로 반환 합니다.
* EdgeNeighborLoader, 간선들에서 이웃 샘플링을 사용하여 하위 그래프(subgraphs)를 `dataframe`, `PyG` 또는 `DGL` 형식으로 반환 합니다.

위의 모든 데이터 로더는 모든 배치를 HTTP 응답(기본값)으로 가져오거나 Kafka를 통해 모든 배치를 스트리밍할 수 있습니다. 전자의 메커니즘은 작은 그래프로 테스트하기에 좋고 빠르지만 데이터 크기 제한이 2GB입니다. 큰 그래프의 경우 크기 제한 및 네트워크 연결 문제로 인해 HTTP 채널이 실패할 수 있습니다. Kafka를 통한 스트리밍은 데이터 견고성과 확장성을 위해 제공됩니다. 또한 Kafka는 다중 소비자(multi-consumer) 사용 사례에 탁월하며 동일한 데이터의 다중 소비자가 있는 경우 모델 검색 또는 하이퍼파라미터 튜닝에 효율적입니다.

데이터 로더는 동종 및 이종 그래프를 모두 지원합니다. 기본적으로 모든 정점 및 간선 유형에서 로드하고 그래프를 동종 그래프로 처리합니다. 그러나 사용자가 로드할 정점 및 간선 유형과 각 유형에서 로드할 속성을 지정할 수도 있습니다. 이 방법으로 사용자는 이기종 그래프 출력을 얻을 수 있습니다.

**NOTE**: 현재 ML Workbench에서 제공하는 모든 기능을 사용하려면 데이터베이스를 한 번만 활성화(activage)해야 합니다. ML Workbench on Cloud를 사용하는 경우 activator가 포함되며 아래 셀을 실행(먼저 주석 해제)하여 활성화(activage)할 수 있습니다. 다른 버전의 Workbench의 경우 https://act.tigergraphlabs.com 에서 활성기(activator)를 다운로드할 수 있습니다. 자세한 지침은 해당 웹사이트에도 포함되어 있습니다.


In [ ]:
# 아래 주석을 제거하고 필요한 정보를 입력하십시오. 자세한 지침은 https://act.tigergraphlabs.com을 참조하십시오.
# !mlwb activate [database address] -u [username] -p [password] -s [secret]

### 데이터베이스 연결 

이 `TigerGraphConnection` 클래스는 TigerGraph 데이터베이스에 대한 연결을 나타냅니다. 후드 아래에는 데이터베이스와 통신하는 데 필요한 정보가 저장됩니다. 꽤 많은 데이터베이스 작업을 수행할 수 있습니다. 자세한 내용은 해당 [문서](https://docs.tigergraph.com/pytigergraph/current/intro/) 를 참조하십시오.

**Note**: 2022년 7월 5일 이후에 생성된 TG 클라우드 DB는 사용자 이름/비밀번호 대신 비밀 번호가 필요합니다. 그렇지 않으면 비워 둘 수 있습니다.

In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # 데이터베이스 서버의 주소로 변경
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # 2022년 7월 5일 이후에 생성된 TG 클라우드 DB에는 user/pass 대신 secret이 필요합니다.  
)

<span style="color:red">아래 셀의 주석을 제거하고 토큰 인증이 활성화된 경우 토큰을 가져오고 설정하기 위해 실행합니다.</span>. 
* 이것은 tgcloud의 모든 데이터베이스에 필요합니다.
* `<secret>`은 사용자 암호입니다. 자세한 내용은 https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets 를 참조하세요.
* secret을 모르는 경우  `secret=conn.createSecret()`를 사용하여 secret을 만들 수 있습니다.

In [ ]:
#conn.getToken(<secret>)

In [3]:
# 모든 정점 유형에 대한 정점 수
conn.getVertexCount('*')

{'Paper': 2708}

In [4]:
# 모든 간선 유형에 대한 간선 수
conn.getEdgeCount()

{'Cite': 10556}

### Edge Neighbor Loader

`EdgeNeighborLoader`는 다음과 같은 방식으로 그래프의 모든 에지에서 인접 샘플링을 일괄적으로 수행합니다.

* 에지의 지정된 수(`batch_size`)를 시드로 선택합니다. 배치 수는 총 정점 수를 배치 크기로 나눈 값입니다.
  * 대신 배치 개수(`num_batches`)를 지정하면 총 에지 개수를 배치 개수로 나누어 `batch_size`를 계산합니다.
  * 두 매개변수를 모두 지정하면 `batch_size`가 우선합니다.
* 시드(seed) 에지에 붙은 정점에서 시작하여 각 정점의 이웃의 지정된 수(`num_neighbors`)를 무작위로 선택합니다.
* 각 인접 항목에 대해 동일한 수의 인접 항목을 선택하고 지정된 수의 홉(`num_hops`) 수행을 완료할 때까지 이 프로세스를 반복합니다.
        
이것은 하나의 하위 그래프를 생성합니다. 이 데이터 로더를 반복할 때 모든 에지는 어느 시점에서 시드로 선택되고 시드에서 하위 그래프가 확장됩니다. 시드를 특정 에지로 제한하려면 `filter_by`에 제공된 부울 속성을 사용하여 시드로 포함할 수 있는 에지를 나타냅니다. 특정 유형의 정점 및 에지에서 로드하려면 `v_in_feats`, `v_out_labels`, `v_extra_feats`, `e_in_feats`, `e_out_labels`, `e_extra_feats`에 대해 `dict` 입력을 사용하세요. 여기서 dict의 키는 정점입니다. 또는 선택할 정점 유형 및 값은 정점 또는 정점 유형에서 수집할 속성 목록입니다.

NOTE: 그래프에서 로더를 처음 초기화하는 경우 데이터베이스에 해당 쿼리를 설치하므로 초기화에 1분이 소요될 수 있습니다. 하지만 쿼리 설치는 한 번만 하면 되므로 같은 그래프에서 다시 로더를 초기화해도 시간이 걸리지 않습니다.


인수(Args):
* v_in_feats (list or dict, optional):
        입력 기능(feature)으로 사용할 정점 속성입니다.
        목록인 경우 모든 정점 유형의 목록에 있는 속성이 선택됩니다. 
        특정 속성이 모든 정점 유형에 존재하지 않으면 오류가 발생합니다. 
        dict인 경우 dict의 키는 선택할 정점 유형이고 값은 각 정점 유형에 대해 
        선택해야 하는 속성의 목록입니다. 숫자 및 부울 속성만 허용됩니다. 
        속성 유형은 데이터베이스 스키마에서 자동으로 결정됩니다. 기본값은 없음입니다.
* v_out_labels (list or dict, optional):
        예측을 위한 레이블로 사용할 정점 속성입니다. 
        목록인 경우 모든 정점 유형의 목록에 있는 속성이 선택됩니다. 
        특정 속성이 모든 정점 유형에 존재하지 않으면 오류가 발생합니다. 
        dict인 경우 dict의 키는 선택할 정점 유형이고 값은 각 정점 유형에 대해 
        선택해야 하는 속성의 목록입니다. 숫자 및 부울 속성만 허용됩니다. 기본값은 없음입니다.
* v_extra_feats (list or dict, optional):
        학습/테스트 데이터의 지표와 같이 가져올 기타 속성입니다. 
        목록인 경우 모든 정점 유형의 목록에 있는 속성이 선택됩니다. 
        특정 속성이 모든 정점 유형에 존재하지 않으면 오류가 발생합니다. 
        dict인 경우 dict의 키는 선택할 정점 유형이고 값은 각 정점 유형에 대해 
        선택해야 하는 속성의 목록입니다. 모든 유형의 속성이 허용됩니다. 기본값은 없음입니다.
* e_in_feats (list or dict, optional):
        입력 기능(feature)으로 사용할 에지 속성입니다. 
        목록인 경우 모든 에지 유형의 목록에 있는 속성이 선택됩니다. 
        특정 속성이 모든 에지 유형에 존재하지 않는 경우 오류가 발생합니다. 
        dict인 경우 dict의 키는 선택해야 하는 에지 유형이고 값은 에지 유형별로 
        선택해야 하는 속성의 목록입니다. 숫자 및 부울 속성만 허용됩니다. 
        속성 유형은 데이터베이스 스키마에서 자동으로 결정됩니다. 기본값은 없음입니다.
* e_out_labels (list or dict, optional):
        예측을 위한 레이블로 사용할 에지 속성입니다. 
        목록인 경우 모든 에지 유형의 목록에 있는 속성이 선택됩니다. 
        특정 속성이 모든 에지 유형에 존재하지 않는 경우 오류가 발생합니다. 
        dict인 경우 dict의 키는 선택해야 하는 에지 유형이고 값은 에지 유형별로 
        선택해야 하는 속성의 목록입니다. 
        숫자 및 부울 속성만 허용됩니다. 기본값은 없음입니다.
* e_extra_feats (list or dict, optional):
        훈련/테스트 데이터의 지표와 같이 얻을 다른 에지 속성. 
        목록인 경우 모든 에지 유형의 목록에 있는 속성이 선택됩니다. 
        특정 속성이 모든 에지 유형에 존재하지 않는 경우 오류가 발생합니다. 
        dict인 경우 dict의 키는 선택해야 하는 에지 유형이고 값은 에지 유형별로 
        선택해야 하는 속성의 목록입니다. 모든 유형의 속성이 허용됩니다. 
        기본값은 없음입니다.
* batch_size (int, optional):  
        각 배치의 정점 수입니다. 기본값은 없음입니다.
* num_batches (int, optional): 
        에지를 분할할 배치 수입니다. 기본값은 1입니다.
* num_neighbors (int, optional):
        각 정점에 대해 샘플링할 이웃 수입니다. 기본값은 10입니다.
* num_hops (int, optional):
        이웃을 샘플링할 때 통과할 홉 수입니다. 기본값은 2입니다.
* shuffle (bool, optional):  
        데이터를 로드하기 전에 에지를 섞을지 여부입니다. 기본값은 False입니다.
* filter_by (str, optional):
        포함된 에지를 나타내는 데 사용되는 부울 속성입니다. 기본값은 없음입니다.
* output_format (str, optional):
        로더의 출력 데이터 형식입니다. "데이터 프레임"만 지원됩니다. 
        기본값은 "데이터 프레임"입니다.
* loader_id (str, optional):
        임의의 문자열이 될 수 있는 로더의 식별자입니다.
        Kafka 주제 이름으로도 사용됩니다. 
        `None`이면 임의의 문자열이 생성됩니다. 기본값은 없음입니다.
* buffer_size (int, optional):
        메모리에 프리페치하고 저장할 데이터 배치 수입니다. 기본값은 4입니다.
* timeout (int, optional):
        GSQL 쿼리에 대한 시간 초과 값(ms)입니다. 기본값은 300000입니다.

#### http를 통해 하위 그래프 가져오기 

In [6]:
%%time
neighbor_loader = conn.gds.edgeNeighborLoader(
    num_batches=10,
    num_neighbors = 10,
    num_hops =2,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=False,
    filter_by=None
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 213 ms, sys: 42.3 ms, total: 255 ms
Wall time: 45.3 s


In [7]:
%%time
for i, batch in enumerate(neighbor_loader):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
Data(edge_index=[2, 7995], edge_feat=[7995], is_train=[7995], is_val=[7995], is_seed=[7995], x=[2337, 1433], y=[2337], train_mask=[2337], val_mask=[2337], test_mask=[2337])
----Batch 1----
Data(edge_index=[2, 8560], edge_feat=[8560], is_train=[8560], is_val=[8560], is_seed=[8560], x=[2435, 1433], y=[2435], train_mask=[2435], val_mask=[2435], test_mask=[2435])
----Batch 2----
Data(edge_index=[2, 8774], edge_feat=[8774], is_train=[8774], is_val=[8774], is_seed=[8774], x=[2432, 1433], y=[2432], train_mask=[2432], val_mask=[2432], test_mask=[2432])
----Batch 3----
Data(edge_index=[2, 8855], edge_feat=[8855], is_train=[8855], is_val=[8855], is_seed=[8855], x=[2450, 1433], y=[2450], train_mask=[2450], val_mask=[2450], test_mask=[2450])
----Batch 4----
Data(edge_index=[2, 8574], edge_feat=[8574], is_train=[8574], is_val=[8574], is_seed=[8574], x=[2428, 1433], y=[2428], train_mask=[2428], val_mask=[2428], test_mask=[2428])
----Batch 5----
Data(edge_index=[2, 7940], edge_feat=[7

#### 이종(heterogeneous) 그래프의 경우

`Cora`는 동종(homogeneous) 그래프 이므로 다른 그래프에 연결하여 이종(demostrate) 그래프의 사용 사례를 시연합니다.

In [8]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # 데이터베이스 서버의 주소로 변경
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # 2022년 7월 5일 이후에 생성된 TG 클라우드 DB에는 user/pass 대신 secret이 필요합니다.   
)

# 토큰 인증이 활성화된 경우 토큰을 가져오고 설정하려면 아래 주석을 제거하십시오.
#conn.getToken(<secret>) # <secret>은 사용자 암호입니다. 자세한 내용은 https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets를 참조하세요.

In [9]:
print(conn.gsql("ls"))

---- Graph hetero
Vertex Types:
- VERTEX v0(PRIMARY_ID id INT, x LIST<DOUBLE>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v1(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v2(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE v0v0(FROM v0, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v1(FROM v1, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v2(FROM v1, TO v2, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v0(FROM v2, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v1(FROM v2, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v2(FROM v2, TO v2, is_train BOOL, is_val BOOL)

Graphs:
- Graph hetero(v0:v, v1:v, v2:v, v0v0:e, v1v1:e, v1v2:e, v2v0:e,

In [10]:
%%time
neighbor_loader = conn.gds.edgeNeighborLoader(
    v_in_feats={"v0": ["x"],
                "v1": ["x"],
                "v2": ["x"]},
    v_out_labels={"v0": ["y"]},
    v_extra_feats={"v0": ["train_mask", "val_mask", "test_mask"]},
    e_extra_feats={"v0v0": ["is_train"], 
                   "v2v0": ["is_train"],
                   "v1v2": ["is_train"]},
    num_batches=16,
    num_neighbors=10,
    num_hops=2,
    shuffle=False,
    output_format="PyG",
    add_self_loop=False,
    loader_id=None,
    buffer_size=4,
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 26.8 ms, sys: 5.07 ms, total: 31.8 ms
Wall time: 45.9 s


In [11]:
%%time
for i, batch in enumerate(neighbor_loader):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
HeteroData(
  v0={
    x=[76, 77],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[46, 57] },
  v2={ x=[100, 48] },
  (v0, v0v0, v0)={
    edge_index=[2, 663],
    is_train=[663],
    is_seed=[663]
  },
  (v1, v1v2, v2)={
    edge_index=[2, 400],
    is_train=[400],
    is_seed=[400]
  },
  (v2, v2v0, v0)={
    edge_index=[2, 884],
    is_train=[884],
    is_seed=[884]
  }
)
----Batch 1----
HeteroData(
  v0={
    x=[76, 77],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[47, 57] },
  v2={ x=[99, 48] },
  (v1, v1v2, v2)={
    edge_index=[2, 408],
    is_train=[408],
    is_seed=[408]
  },
  (v2, v2v0, v0)={
    edge_index=[2, 872],
    is_train=[872],
    is_seed=[872]
  },
  (v0, v0v0, v0)={
    edge_index=[2, 655],
    is_train=[655],
    is_seed=[655]
  }
)
----Batch 2----
HeteroData(
  v0={
    x=[76, 77],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[

### Kafka를 통한 스트리밍 

**Note**: Kafka 스트리밍 기능은 Enterprise Edition에서만 사용할 수 있습니다. 사용하려면 Enterprise Edition을 활성화해야 합니다.

In [ ]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # 데이터베이스 서버의 주소로 변경
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # 2022년 7월 5일 이후에 생성된 TG 클라우드 DB에는 user/pass 대신 secret이 필요합니다.   
)

# 토큰 인증이 활성화된 경우 토큰을 가져오고 설정하려면 아래 주석을 제거하십시오.
#conn.getToken(<secret>) # <secret>은 사용자 암호입니다. 자세한 내용은 https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets를 참조하세요.

#### 카프카(Kafka) 설정 
여기에서 Kafka를 설정합니다. 구성되면 설정이 새로 생성된 모든 데이터 로더와 공유되며 각 로더에 대해 Kafka를 설정할 필요가 없습니다. 자세한 설정 은 공식 [문서](https://docs.tigergraph.com/pytigergraph/current/gds/gds#_configurekafka) 를 참조하십시오.

In [ ]:
conn.gds.configureKafka(
    kafka_address="127.0.0.1:9092",
    kafka_security_protocol="SASL_PLAINTEXT",
    kafka_sasl_mechanism="PLAIN",
    kafka_sasl_plain_username="your username",
    kafka_sasl_plain_password="your password"
)

#### 하위 그래프(subgraphs) 가져오기

In [ ]:
%%time
neighbor_loader = conn.gds.EdgeNeighborLoader(
    num_batches=10,
    num_neighbors = 10,
    num_hops =2,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=True,
    filter_by=None
)

In [ ]:
%%time
for i, batch in enumerate(neighbor_loader):
    print("----Batch {}----".format(i))
    print(batch)